In [13]:
from google.cloud import bigquery
import json

In [14]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="gcp_api_keys.json"

In [15]:
def verify_auth():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    return buckets

print(verify_auth())

[]


In [61]:
client = bigquery.Client()

query = """
    SELECT *
    FROM `fh-bigquery.hackernews.stories`
    LIMIT 20
"""

query = """
    SELECT *
    FROM fh-bigquery.hackernews.stories
    WHERE
        DEAD IS NOT true
        AND TITLE != "None"
    LIMIT 1000
"""

query = """
    SELECT *
    FROM fh-bigquery.hackernews.stories s
    TABLESAMPLE SYSTEM (1 PERCENT)
    WHERE
        s.DEAD IS NOT true
        AND s.TITLE != "None"
        AND NOT (s.url = "NONE" AND s.text = "None")
        AND rand() < 0.1
"""

query_join = """
    SELECT *
    FROM
        fh-bigquery.hackernews.stories s,
        fh-bigquery.hackernews.comments c
    TABLESAMPLE SYSTEM (5 PERCENT)
    WHERE
        c.parent = s.id
        AND s.DEAD IS NOT true
        AND s.TITLE != "None"
        AND NOT (s.url = "NONE" AND s.text = "None")
        
        AND c.DEAD IS NOT true
        AND c.text != "None"

        AND rand() < 0.1
"""

query_join_2 = """
    SELECT "story" as type, s.id, s.by, s.score, s.time, s.time_ts, s.title, s.url, s.text, null as parent, s.deleted, s.dead, s.descendants, s.author
    FROM fh-bigquery.hackernews.stories s
    WHERE
        s.DEAD IS NOT true
        AND s.TITLE != "None"
        AND NOT (s.url = "NONE" AND s.text = "None")
        AND rand() < 0.1
        
    UNION ALL
    
    SELECT "comment" as type, c.id, c.by, c.ranking, c.time, null as time_ts, null as title, null as url, c.text, c.parent, c.deleted, c.dead, null as descendants, c.author
    FROM fh-bigquery.hackernews.comments c
    TABLESAMPLE SYSTEM (5 PERCENT)
    WHERE
        c.DEAD IS NOT true
        AND c.text != "None"
        
        AND c.parent in (
            SELECT id FROM fh-bigquery.hackernews.stories s2
            WHERE
                s2.DEAD IS NOT true
                AND s2.TITLE != "None"
                AND NOT (s2.url = "NONE" AND s2.text = "None")
                AND rand() < 0.1
        )

        AND rand() < 0.1
"""

query_join_3 = """
    WITH Stories AS
    (
        SELECT "story" as type, s.id, s.by, s.score, s.time, s.time_ts, s.title, s.url, s.text, null as parent, s.deleted, s.dead, s.descendants, s.author
        FROM fh-bigquery.hackernews.stories s
        WHERE
            s.DEAD IS NOT true
            AND s.TITLE != "None"
            AND NOT (s.url = "NONE" AND s.text = "None")
            AND rand() < 0.1
    )
    SELECT * from Stories
    
    UNION ALL
    
    SELECT "comment" as type, c.id, c.by, c.ranking, c.time, null as time_ts, null as title, null as url, c.text, c.parent, c.deleted, c.dead, null as descendants, c.author
    FROM fh-bigquery.hackernews.comments c
    TABLESAMPLE SYSTEM (1 PERCENT)
    WHERE
        c.DEAD IS NOT true
        AND c.text != "None"
        
        AND c.parent in (SELECT id FROM Stories)

        AND rand() < 0.1
"""

query_job = client.query(query_join) # Make an API request.

# print("The query data:")
# for row in query_job:
#     print(row)
    
json_obj = []
print("starting to write data from", query_job)
with open("small_data_random.txt", "w") as f:
    for index, row in enumerate(query_job):
        row_dict = dict(row)
        del row_dict['time_ts']
        if 'time_ts_1' in row_dict:
            del row_dict['time_ts_1']
        f.write(json.dumps(row_dict))
        f.write("\n")
        if index % 1000 == 0:
            print("Wrote", index + 1)

starting to write data from QueryJob<project=famous-mission-334512, location=US, id=60346a03-03dd-4744-82b4-2be660e54514>
Wrote 1
Wrote 1001
Wrote 2001
Wrote 3001
Wrote 4001
Wrote 5001
Wrote 6001
Wrote 7001
Wrote 8001
Wrote 9001
Wrote 10001
Wrote 11001


In [39]:
from pyspark import SparkConf, SparkContext
from collections import defaultdict
import numpy as np
import json
import datetime

In [35]:
# import findspark
# os.unsetenv("SPARK_HOME")
# # os.environ['SPARK_HOME'] = "/Users/sergiuiacob/.pyenv/shims/pyspark"
# findspark.init()

In [36]:
! javac -version

# MUST ONLY BE RUN ONCE
if 'sc' not in locals():
    conf = SparkConf().setMaster("local").setAppName("BDA Hacker News")
    sc = SparkContext(conf = conf)

javac 9.0.4


In [75]:
myRDD = sc.textFile("small_data_random.txt").map(json.loads)

In [76]:
myRDD.take(1)

[{'id': 1033400,
  'by': 'kogir',
  'score': 0,
  'time': 1401561740,
  'title': 'Placeholder',
  'url': '',
  'text': 'Mind the gap.',
  'deleted': None,
  'dead': None,
  'descendants': 0,
  'author': 'kogir'}]

In [89]:
mapRes = myRDD.map(lambda row: (datetime.datetime.fromtimestamp(row['time']).hour, row['score']))

In [90]:
redResult = mapRes.groupByKey().mapValues(lambda vals: np.mean(list(vals)))

In [99]:
results = redResult.collect()

results.sort(key = lambda key: key[1])
results[::-1]

[(5, 20.413857677902623),
 (18, 16.154589371980677),
 (20, 14.377813504823152),
 (3, 14.182724252491694),
 (4, 13.770833333333334),
 (6, 13.686475409836065),
 (22, 13.641346153846154),
 (21, 13.494290375203915),
 (13, 13.484375),
 (16, 13.329138431752177),
 (11, 13.165289256198347),
 (17, 13.019064124783363),
 (19, 12.843059936908517),
 (2, 12.741486068111454),
 (15, 12.59046454767726),
 (0, 12.46979865771812),
 (10, 12.02247191011236),
 (1, 11.810771470160116),
 (14, 11.36687306501548),
 (8, 11.162666666666667),
 (23, 11.139691714836223),
 (7, 10.817813765182187),
 (12, 10.487179487179487),
 (9, 9.559036144578313)]